# Interactive Componentes

## Single Callbacks for Interactivity

- Steps to create a callback from interactions:
    - Create a function to return some desired output;
    - Decorate that function with an @app.callback decorator;
        - Set an **Output** to a component id;
        - Set an **Input** to a component id.
    - Connect the desired properties.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [6]:
app = dash.Dash()

app.layout = html.Div(
    [
        dcc.Input(
            id = 'my-id',
            value = 'Initial Text',
            type = 'text',
        ),
        html.Div(
            id = 'my-div',
            style = {
                'border': '2px blue solid'
            }
        )
    ]
)

@app.callback(
    Output(
        component_id = 'my-div', 
        component_property = 'children'
    ),
    [
        Input(
            component_id = 'my-id',
            component_property = 'value'
        )
    ]
)
def update_output_div(input_value):
    return "Your entered: {}".format(input_value)

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jun/2022 16:39:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:11] "POST /_dash-update-component

## Dash Callbacks for Graphs